&emsp;&emsp;这是一份从美国亚马逊网站上爬取的浴帘数据。

# 数据处理

&emsp;&emsp;导入数据：

In [145]:
import pandas as pd
import numpy as np
from pyecharts.charts import Bar, Line, Grid, Pie, HeatMap
from pyecharts import options as opts
from pyecharts.globals import ThemeType
import matplotlib.pyplot as plt

df = pd.read_excel('./美国浴帘2021.1.20.xlsx')

&emsp;&emsp;查看数据的基本信息：

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   编号      49 non-null     int64         
 1   商品链接    49 non-null     object        
 2   标题      49 non-null     object        
 3   商标      49 non-null     object        
 4   Asin    49 non-null     object        
 5   销量      47 non-null     float64       
 6   价格      49 non-null     object        
 7   上架时间    49 non-null     datetime64[ns]
 8   评分      49 non-null     float64       
 9   评价数量    49 non-null     int64         
 10  评级      49 non-null     int64         
 11  评分.1    49 non-null     int64         
 12  材料      49 non-null     object        
dtypes: datetime64[ns](1), float64(2), int64(4), object(6)
memory usage: 5.1+ KB


&emsp;&emsp;可以看到：销量只有47个非空值，其他变量均有49个非空值，表明销量数据存在空值，需要剔除。

In [3]:
data = df.dropna()

&emsp;&emsp;数据类型也需要进行相应的转换：价格变量的类型需要转换为 float。

In [28]:
# 价格变量中有 None 值，直接转换会出错
data.loc[:,'价格'] = data.loc[:,'价格'].replace('None', 0).astype('float64') 

&emsp;&emsp;进一步筛选价格不等于 0 的数据：

In [5]:
data = data[data['价格'] != 0] 

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46 entries, 0 to 48
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   编号      46 non-null     int64         
 1   商品链接    46 non-null     object        
 2   标题      46 non-null     object        
 3   商标      46 non-null     object        
 4   Asin    46 non-null     object        
 5   销量      46 non-null     float64       
 6   价格      46 non-null     float64       
 7   上架时间    46 non-null     datetime64[ns]
 8   评分      46 non-null     float64       
 9   评价数量    46 non-null     int64         
 10  评级      46 non-null     int64         
 11  评分.1    46 non-null     int64         
 12  材料      46 non-null     object        
dtypes: datetime64[ns](1), float64(3), int64(4), object(5)
memory usage: 5.0+ KB


&emsp;&emsp;再来看一下，数据的各个变量都代表什么内容：

- 编号：无效信息，由爬虫人员生成，无实际意义；
- 商品链接：无效信息，表示商品的亚马逊链接，可用于核查数据是否准确；
- 标题：由商标等信息组成，可能从中拆分出有用的信息，需要进一步分析；
- 商标：表示商品的品牌名；
- [Asin](https://zhuanlan.zhihu.com/p/214132694)：无效信息，由亚马逊根据 UPC 代码自动生成的唯一商品标识代码；
- 销量：有效信息，表示商品的销售数量；
- 价格：有效信息，表示商品的销售价格；
- 上架时间：有效信息，可结合其他信息，比如销售数量，计算日销量等。
- 评分：有效信息，表示买家对商品的喜爱度；
- 评价数量：有效信息，评价数量小于销售数量，可用于计算评价率等指标；
- 评级：目前并不清楚它代表的信息，需要结合业务进行了解；
- 评分.1：目前并不清楚它代表的信息，需要结合业务进行了解；
- 材料：有效信息，表示商品的制造材料。

&emsp;&emsp;接下来可以简单看一下数据的分布情况：

In [7]:
data.describe()

,编号,销量,价格,评分,评价数量,评级,评分.1
count,46.000000,46.000000,46.000000,46.00000,46.000000,46.000000,46.000000
mean,25.413043,3807.391304,19.935870,4.61087,3086.239130,414.260870,74.565217
std,14.966000,6335.532522,6.323935,0.16361,6526.992413,624.473376,130.761131
min,1.000000,570.000000,8.280000,4.10000,37.000000,0.000000,0.000000
25%,12.250000,1335.000000,14.990000,4.50000,934.000000,38.500000,4.000000
50%,25.500000,1635.000000,19.990000,4.60000,1881.000000,93.500000,15.500000
75%,37.750000,3502.500000,23.865000,4.70000,2913.750000,423.750000,55.250000
max,50.000000,38910.000000,39.990000,4.80000,44610.000000,2341.000000,471.000000


&emsp;&emsp;可以看出：销量、价格、评分的分布比较合理，极值对平均值的影响不大，评级数量则不同，极值的存在影响了平均值。

# 销售量与销售额

&emsp;&emsp;先计算销售额：$销售额=销量 \times 价格$

In [8]:
data['销售额'] = data['销量']*data['价格']

&emsp;&emsp;再利用数据透视表计算各品牌的销量和销售额：

In [119]:
sale_data = data.pivot_table(index='商标',
                           values=['销量', '销售额', '评分'],
                           aggfunc={
                               '销量': 'sum',
                               '销售额': 'sum',
                               '评分': 'mean'
                           }).round(1)

&emsp;&emsp;再计算累计销量占比和累计销售额占比：

In [25]:
sale_cum_data = sale_data.sort_values('销量', ascending=False).apply(lambda x: x.cumsum()/x.sum()).round(2)

&emsp;&emsp;进行可视化展示：

In [179]:
bar = (Bar()
       .add_xaxis(sale_data.sort_values('销量', ascending=False).index.tolist())
       .add_yaxis(
            "销量", 
            sale_data.sort_values('销量', ascending=False)['销量'].tolist(),
           yaxis_index=0
       )
       .extend_axis(yaxis=opts.AxisOpts(position="right"))
       .set_global_opts(
           title_opts=opts.TitleOpts(
               title="各品牌浴帘的销量", 
               pos_left="left"
           ),
           xaxis_opts=opts.AxisOpts(
               axistick_opts=opts.AxisTickOpts(is_align_with_label=True),
               axislabel_opts=opts.LabelOpts(rotate=-90)
           )
       )
       .set_series_opts(label_opts=opts.LabelOpts(is_show=False, position='left'))
       )

line = (
    Line()
    .add_xaxis(sale_data.sort_values('销量', ascending=False).index.tolist())
    .add_yaxis(
        "累计销量占比",
        sale_cum_data['销量'].tolist(),
        yaxis_index=1,
        label_opts=opts.LabelOpts(is_show=False, position='left'),
    )
)

bar.overlap(line)
grid = Grid(init_opts=opts.InitOpts(theme=ThemeType.VINTAGE))
grid.add(bar, opts.GridOpts(pos_left="10%", pos_right="10%", pos_bottom="25%"), is_control_axis_index=True)
grid.render_notebook()

&emsp;&emsp;可以看出：
1. AmazerrBath 品牌的浴帘销量最高且明显高于其他品牌;
2. 从累计销量占比来看，前三个品牌的浴帘销量占比超过50%。

In [225]:
bar = (Bar()
       .add_xaxis(sale_data.sort_values('销售额', ascending=False).index.tolist())
       .add_yaxis(
            "销售额", 
            sale_data.sort_values('销售额', ascending=False)['销售额'].tolist(),
           yaxis_index=0
       )
       .extend_axis(yaxis=opts.AxisOpts(position="right"))
       .set_global_opts(
           title_opts=opts.TitleOpts(
               title="各品牌浴帘的销售额", 
               pos_left="left"
           ),
           xaxis_opts=opts.AxisOpts(
               axistick_opts=opts.AxisTickOpts(is_align_with_label=True),
           axislabel_opts=opts.LabelOpts(rotate=-90)
           )
       )
       .set_series_opts(label_opts=opts.LabelOpts(is_show=False, position='left'))
       )

line = (
    Line()
    .add_xaxis(sale_data.sort_values('销售额', ascending=False).index.tolist())
    .add_yaxis(
        "累计销售额占比",
        sale_cum_data['销售额'].tolist(),
        yaxis_index=1,
        label_opts=opts.LabelOpts(is_show=False, position='left'),
    )
)

bar.overlap(line)
grid = Grid(init_opts=opts.InitOpts(theme=ThemeType.VINTAGE))
grid.add(bar, opts.GridOpts(pos_left="10%", pos_right="10%", pos_bottom="25%"), is_control_axis_index=True)
grid.render_notebook()

&emsp;&emsp;同样可以看出：
1. AmazerrBath 品牌的浴帘销售额最高且明显高于其他品牌;
2. 从累计销量占比来看，前四个品牌的浴帘销售额占比超过50%。

In [13]:
sale_data.corr()

,销售额,销量
销售额,1.000000,0.970239
销量,0.970239,1.000000


&emsp;&emsp;销售额与销量的相关性也符合规律。

# 销售周期

&emsp;&emsp;先计算上架天数：

In [35]:
data['上架天数'] = (pd.Timestamp('2021-1-22') - data['上架时间'])/np.timedelta64(1, 'D')

&emsp;&emsp;再对上架天数进行分段统计：

In [66]:
data['上架天数'].describe()

count      46.000000
mean      692.673913
std       701.470077
min        15.000000
25%       303.250000
50%       524.000000
75%       719.250000
max      3574.000000
Name: 上架天数, dtype: float64

In [71]:
shelves_data = pd.cut(
    data['上架天数'], [
        0,
        100,
        300,
        600,
        1000,
        100000
    ],
    labels=['100天以下', '100~300天', '300~600天', '600~1000天', '1000天以上']).value_counts()

In [74]:
histogram = (
    Bar(init_opts=opts.InitOpts(theme=ThemeType.VINTAGE))
    .add_xaxis(shelves_data.index.tolist())
    .add_yaxis("", shelves_data.values.tolist(), category_gap=0)
    .set_global_opts(title_opts=opts.TitleOpts(title="上架天数分布", pos_left='center'))
    .render_notebook()
)

histogram

&emsp;&emsp;可以看出：大部分商品的上架天数均大于300天，表明各商品销售时间均比较长，商品样式更换速度慢。

In [83]:
data['上架月份'] = data['上架时间'].map(lambda x: x.month)

In [84]:
shelves_month_data = data.groupby('上架月份')['上架时间'].count()

In [87]:
bar = (
    Bar(init_opts=opts.InitOpts(theme=ThemeType.VINTAGE))
    .add_xaxis(shelves_month_data.index.tolist())
    .add_yaxis("", shelves_month_data.tolist())
    .set_global_opts(title_opts=opts.TitleOpts(title="上架月份分布", pos_left='center'))
    .render_notebook()
)

bar

&emsp;&emsp;可以看出：年末商品上架数量比较少，多集中在5月、8月上架新商品。

# 价格分布

In [89]:
data['价格'].describe()

count    46.000000
mean     19.935870
std       6.323935
min       8.280000
25%      14.990000
50%      19.990000
75%      23.865000
max      39.990000
Name: 价格, dtype: float64

In [90]:
price_data = pd.cut(
    data['价格'], [
        0,
        10,
        15,
        20,
        25,
        30,
        35, 
        40
    ],
    labels=['0~10', '10~15', '15~20', '20~25', '25~30', '30~35', '35~40']).value_counts()

In [98]:
bar = (
    Bar(init_opts=opts.InitOpts(theme=ThemeType.VINTAGE))
    .add_xaxis(price_data.index.tolist())
    .add_yaxis("", price_data.tolist(), category_gap=0)
    .set_global_opts(
        title_opts=opts.TitleOpts(title="浴帘价格分布", pos_left='center'),
        legend_opts=opts.LegendOpts(is_show=False),
        xaxis_opts=opts.AxisOpts(name="单位：美元")
    )
    .render_notebook()
)

bar

&emsp;&emsp;可以看出：浴帘价格集中在 10\~25 美元，其中 20\~25 美元区间内的商品数量最多。

# 品牌价格区间与销量

In [141]:
data['价格区间'] = pd.cut(
    data['价格'], [
        0,
        10,
        15,
        20,
        25,
        30,
        35, 
        40
    ],
    labels=['0~10', '10~15', '15~20', '20~25', '25~30', '30~35', '35~40'])

In [143]:
price_sale_data = data.pivot_table(
    index = '商标',
    columns = '价格区间',
    values = '销量',
    aggfunc=np.sum
)

In [168]:
value = [[j, i, price_sale_data.iloc[i, j]]
         for i in range(len(price_sale_data.index))
         for j in range(len(price_sale_data.columns))]

In [221]:
heatmap = (
    HeatMap()
    .add_xaxis(price_sale_data.columns.tolist())
    .add_yaxis(
        "", 
        price_sale_data.index.tolist(), 
        value=value, 
        label_opts=opts.LabelOpts(is_show=True, position="inside")
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="浴帘品牌价格区间与销量", pos_left = 'center'),
        xaxis_opts=opts.AxisOpts(name="单位：美元", axislabel_opts={"interval":"-10"}),
        visualmap_opts=opts.VisualMapOpts(min_=500, max_=45000)
    )
)

grid = Grid(init_opts=opts.InitOpts(theme=ThemeType.VINTAGE))
grid.add(heatmap, opts.GridOpts(pos_left='20%', pos_top='5%', pos_bottom='5%'), is_control_axis_index=True)
grid.render_notebook()

&emsp;&emsp;结论：10~15美元的浴帘销量最高，为 68,130，其中 AmazerBath 品牌的浴帘销量最高，占比约为64%。

In [228]:
price_sale_data.sum()
(price_sale_data.sum())/(price_sale_data.sum().sum())
price_sale_data.loc['AmazerBath', '10~15']/price_sale_data.loc[:, '10~15'].sum()

价格区间
0~10      1110.0
10~15    68130.0
15~20    55127.0
20~25    38803.0
25~30     6060.0
30~35     4500.0
35~40     1410.0
dtype: float64

价格区间
0~10     0.006338
10~15    0.389003
15~20    0.314760
20~25    0.221554
25~30    0.034601
30~35    0.025694
35~40    0.008051
dtype: float64

0.637604579480405

# 品牌的评分

In [182]:
bar = (
    Bar()
    .add_xaxis(sale_data.index.tolist())
    .add_yaxis(
        "", 
        sale_data['评分'].tolist(),
        markline_opts=opts.MarkLineOpts(
               data=[opts.MarkLineItem(type_='average', name='平均值')]
           )
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="浴帘品牌评分", pos_left='center'),
        yaxis_opts=opts.AxisOpts(min_ = 4),
        xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=-90))
    )
)

grid = Grid(init_opts=opts.InitOpts(theme=ThemeType.VINTAGE))
grid.add(bar, opts.GridOpts(pos_bottom="25%"), is_control_axis_index=True)
grid.render_notebook()

&emsp;&emsp;结论：浴帘商品的平均评分为4.59，有 18 个商品的评分超过平均评分（总数为27个）。

In [124]:
sale_data[['销量', '评分']].corr()

,销量,评分
销量,1.000000,0.168746
评分,0.168746,1.000000


&emsp;&emsp;从销量与评分的相关系数来看，评分对销量的影响相对比较小。

# 商品的评价数量

In [126]:
data['评价数量'].describe()

count       46.000000
mean      3086.239130
std       6526.992413
min         37.000000
25%        934.000000
50%       1881.000000
75%       2913.750000
max      44610.000000
Name: 评价数量, dtype: float64

In [131]:
score_num_data = pd.cut(
    data['评价数量'], [
        0,
        100,
        500,
        1000,
        2000,
        3000,
        1000000
    ],
    labels=['0~100', '100~500', '500~1000', '1000~2000', '2000~3000', '3000以上']).value_counts()

In [139]:
pie = (
    Pie(init_opts=opts.InitOpts(theme=ThemeType.VINTAGE))
    .add("", [list(i) for i in zip(score_num_data.index.tolist(), score_num_data.values.tolist())])
    .set_global_opts(
        title_opts=opts.TitleOpts(title="浴帘评价数分布", pos_left="center"),
        legend_opts=opts.LegendOpts(is_show=False)
    )
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {d}%"))
    .render_notebook()
)
pie

&emsp;&emsp;结论：浴帘商品的评论数普遍偏多，1000以下的评论数仅占 26.09%。